In [52]:
import rasterio
from rasterio.mask import mask
from shapely.geometry import shape
import fiona
import os

# Directory containing the shapefiles
SHAPEFILE_DIR=r"E:\wenqu\trait_map\trait_map_without_chm\validation\shapefiles\site2c"
# Directory to save the output tifs
OUTPUT_DIR=r'E:\wenqu\trait_map\trait_map_without_chm\validation\clipped_from_aviris_trait_map\sla_ymax'
# Input raster
RASTER_INPUT=r'E:\wenqu\aviris\arctic_region\trait_map\site1b_sla_trait_ymax.tif'

# Get the list of shapefiles
shapefiles = [f for f in os.listdir(SHAPEFILE_DIR) if f.endswith('.shp')]

for shapefile in shapefiles:
    with fiona.open(os.path.join(SHAPEFILE_DIR, shapefile), "r") as shapefile_data:
        # Extract shapefile's geometry
        shapes = [feature["geometry"] for feature in shapefile_data]
        
        with rasterio.open(RASTER_INPUT) as src:
            # Clip the raster with the shapefile's geometry
            out_image, out_transform = mask(src, shapes, crop=True)
            out_meta = src.meta.copy()
            
            # Update the metadata with the clipped raster's details
            out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
            output_name = 'sla_ymax_' + shapefile.replace('.shp', '.tif')
            
            output_raster = os.path.join(OUTPUT_DIR, output_name)
            with rasterio.open(output_raster, "w", **out_meta) as dest:
                dest.write(out_image)

print("Clipping completed!")

Clipping completed!
